In [1]:
from tomok import IFCReader
from tomok import RuleUnitController

In [2]:
reader = IFCReader(ifc_filepath="/mnt/raid6/jaewook133/tomok/acc_server2/uploads/gAAAAABnRUhUhGYAPP9vbIqUPaQSIsf5OcTUKARvnfeyecZN9fcuVZBke-t1bm0GvcJrGzBXpWc_JQquwwuSxJG8jTzPBpt4lydn78pfziXox8CU5vAnwr82w9SQWaIvTmcnFPATrI8N04n7bCgTW6FuH-2JLO_3XG28YHxUDl5cOQetMrDMV64=")

In [3]:
for key in reader.subtype_product_cache.keys():
    print(key)

None
레벨:EXTREMO INICIAL 8 MM
NOTDEFINED
DOOR
HALF_TURN_STAIR
STRAIGHT
LANDING
STRINGER
OPENING
WINDOW
CURTAIN_PANEL
MULLION
FLOOR
STRAIGHT_RUN_STAIR
M_URINARIO-MURO-3D:M_URINARIO-MURO-3D
ROOF
CEILING
COLUMN
BEAM
BASESLAB


In [4]:
reader.subtype_product_cache['FLOOR']

[#58189=IfcSlab('2f$SEp0bT4SfuPGVu0X6np',#18,'바닥:Hormigón 160 mm con losacero 50 mm:371309',$,'바닥:Hormigón 160 mm con losacero 50 mm',#58171,#58188,'371309',.FLOOR.),
 #60671=IfcSlab('3fYD0Vq9XAsewjKrFiZwIE',#18,'바닥:exteriores:477652',$,'바닥:exteriores',#60653,#60670,'477652',.FLOOR.),
 #60705=IfcSlab('3fYD0Vq9XAsewjKrFiZwCM',#18,'바닥:exteriores:478796',$,'바닥:exteriores',#60690,#60704,'478796',.FLOOR.),
 #60729=IfcSlab('3fYD0Vq9XAsewjKrFiZxqZ',#18,'바닥:exteriores:479353',$,'바닥:exteriores',#60714,#60728,'479353',.FLOOR.),
 #60753=IfcSlab('3fYD0Vq9XAsewjKrFiZxzs',#18,'바닥:exteriores:479788',$,'바닥:exteriores',#60738,#60752,'479788',.FLOOR.),
 #60777=IfcSlab('3fYD0Vq9XAsewjKrFiZxxa',#18,'바닥:exteriores:480190',$,'바닥:exteriores',#60762,#60776,'480190',.FLOOR.),
 #60801=IfcSlab('3fYD0Vq9XAsewjKrFiZxWG',#18,'바닥:exteriores:480586',$,'바닥:exteriores',#60786,#60800,'480586',.FLOOR.),
 #60825=IfcSlab('3fYD0Vq9XAsewjKrFiZxkA',#18,'바닥:exteriores:480976',$,'바닥:exteriores',#60810,#60824,'480976',.FLOOR.),


In [5]:
reader.get_products()
e = reader.get_product_by_guid(guid="3fYD0Vq9XAsewjKrFiZqkg")
e

#62866=IfcSlab('3fYD0Vq9XAsewjKrFiZqkg',#18,'바닥:Hormigón 160 mm con losacero 50 mm:501488',$,'바닥:Hormigón 160 mm con losacero 50 mm',#62851,#62865,'501488',.FLOOR.)

In [11]:
e.tree

{'entity_instances': [[62873/IfcRelDefinesByProperties/] [pset: Pset_SlabCommon] [mat: None] [prof: None] [rule:None],
  [300097/IfcRelDefinesByProperties/] [pset: KRPset_KDS 14 20 22_4.11.3 (1)] [mat: None] [prof: None] [rule:None],
  [300119/IfcRelDefinesByProperties/] [pset: KRPset_KDS 14 20 22_4.11.3 (2)] [mat: None] [prof: None] [rule:None],
  [300017/IfcRelDefinesByProperties/] [pset: KRPset_KDS 14 20 22_4.11.1 (3)] [mat: None] [prof: None] [rule:None],
  [286044/IfcRelAssociatesClassification/A10] [pset: Uniformat] [mat: Uniformat] [prof: Uniformat] [rule:None],
  [300083/IfcRelDefinesByProperties/] [pset: KRPset_KDS 14 20 22_4.11.2 (5)] [mat: None] [prof: None] [rule:None],
  [62871/IfcRelDefinesByProperties/] [pset: Pset_EnvironmentalImpactIndicators] [mat: None] [prof: None] [rule:None],
  [284496/IfcRelAssociatesMaterial/] [pset: None] [mat: None] [prof: None] [rule:None],
  [285159/IfcRelDefinesByType/] [pset: None] [mat: None] [prof: None] [rule:None],
  [284440/IfcRelCont

In [8]:
ruleUnitController = RuleUnitController('ruleunits')
ruleUnitController.ruleunits

[]

In [6]:
import pprint
pprint.pprint(ruleUnitController.ruleunits_dict)

{'kds': {'142054': {'040305_01': {'KDS142054_040305_01': <ruleunits.kds.142054.040305_01.KDS142054_040305_01 object at 0x7f1a46a27040>},
                    '040305_03': {'KDS142054_040305_03': <ruleunits.kds.142054.040305_03.KDS142054_040305_03 object at 0x7f1699403a60>},
                    '040305_04': {'KDS142054_040305_04': <ruleunits.kds.142054.040305_04.KDS142054_040305_04 object at 0x7f1699403c10>},
                    '040305_05': {'KDS142054_040305_05': <ruleunits.kds.142054.040305_05.KDS142054_040305_05 object at 0x7f1a46a277f0>}}}}


In [7]:
ruleUnitController['kds']['142054']['040305_01']['KDS142054_040305_01']

In [8]:
from tomok import RuleIFC, IFCReader, Product, RuleUnitController


class RuleIFC_KDS142054_01(RuleIFC):
    # RuleIFC
    def __init__(self,
                 reader: IFCReader,
                 rule_units: RuleUnitController):
        self.reader = reader
        self.runit = {
            '01': rule_units['kds']['142054']['040305_01']['KDS142054_040305_01'],
            '03': rule_units['kds']['142054']['040305_03']['KDS142054_040305_03'],
            '04': rule_units['kds']['142054']['040305_04']['KDS142054_040305_04'],
            '05': rule_units['kds']['142054']['040305_05']['KDS142054_040305_05']
        }

    def retrieve_entities(self,
                          guid: str):
        self.reader.get_products() # IFC 파일에서 부재 목록을 읽어옵니다.
        try:
            target_entity = [self.reader.get_product_by_guid(guid)] # guid를 기반으로 부재를 검색합니다.
        except:
            return []
        return target_entity

    def _get_p_value(self,
                     entity: Product,
                     krpset_name: str,
                     en_name: str):
        try:
            return entity[krpset_name][en_name]
        except:                
            return -1
        
    def process(self,
                entity: Product):
        try:
            # KDS142054_040305_03
            # fOpsecNa = self._get_p_value(entity=entity,
            #                             krpset_name="KRPset_KDS 14 20 54_4.3.5 (3)",
            #                             en_name="Correction factor for attached anchor groups in which eccentricity of tensile force acts")
            fIeprimN = self._get_p_value(entity=entity,
                                        krpset_name="KRPset_KDS 14 20 54_4.3.5 (3)",
                                        en_name="The distance between the resultant force of the tensile force acting on the anchor group subjected to tensile load and the centroid of the anchor group")
            fIcNa = self._get_p_value(entity=entity,
                                      krpset_name="KRPset_KDS 14 20 54_4.3.5 (3)",
                                      en_name="The distance from the center of the anchor to the edge of the projected influence area required to develop the maximum bond strength")
            fOpsecNa = self.runit['03'].correction_factor_for_attached_anchor_groups_with_eccentricity_of_tensile_force(0, fIeprimN, fIcNa)
            # KDS142054_040305_04
            # fOpsedNa = self._get_p_value(entity=entity,
            #                             krpset_name="KRPset_KDS 14 20 54_4.3.5 (4)",
            #                             en_name="Correction factor for edge influence of single anchor anchor or group of anchor anchors")
            fIcamin = self._get_p_value(entity=entity,
                                    krpset_name="KRPset_KDS 14 20 54_4.3.5 (4)",
                                    en_name="Minimum edge distance from anchor shaft center to concrete end")
            fIcNa = self._get_p_value(entity=entity,
                                    krpset_name="KRPset_KDS 14 20 54_4.3.5 (4)",
                                    en_name="The distance from the center of the anchor to the edge of the projected influence area required to develop the maximum bond strength")
            fOpsedNa = self.runit['04'].correction_factor_for_edge_influence_of_single_or_group_of_anchor(0, fIcamin, fIcNa)
            # KDS142054_040305_05
            # fOpscpNa = self._get_p_value(entity=entity,
            #                             krpset_name="KRPset_KDS 14 20 54_4.3.5 (5)",
            #                             en_name="Modification factors for bonded anchors used in uncracked concrete")
            fIcamin = self._get_p_value(entity=entity,
                                    krpset_name="KRPset_KDS 14 20 54_4.3.5 (5)",
                                    en_name="Minimum edge distance from anchor shaft center to concrete end")
            fIcac = self._get_p_value(entity=entity,
                                    krpset_name="KRPset_KDS 14 20 54_4.3.5 (5)",
                                    en_name="risk podium distance")
            fOpscpNa = self.runit['05'].modification_factors_for_bonded_anchors_used_in_uncracked_concrete(0, fIcamin, fIcac)

            # KDS142054_040305_01
            fINa = self._get_p_value(entity=entity,
                                     krpset_name="KRPset_KDS 14 20 54_4.3.5 (1)",
                                     en_name="Nominal bond strength of a single bonded anchor")
            fINag = self._get_p_value(entity=entity,
                                      krpset_name="KRPset_KDS 14 20 54_4.3.5 (1)",
                                      en_name="Nominal bond strength of adhesive anchor group")
            fIANa = self._get_p_value(entity=entity,
                                      krpset_name="KRPset_KDS 14 20 54_4.3.5 (1)",
                                      en_name="Projected influence area of anchor group")
            fIANao = self._get_p_value(entity=entity,
                                       krpset_name="KRPset_KDS 14 20 54_4.3.5 (1)",
                                       en_name="Projected influence area of a single anchor")
            fIpsedNa = fOpsedNa # KDS142054_040305_04 실행 결과
            fIpscpNa = fOpscpNa # KDS142054_040305_05 실행 결과
            fINba = self._get_p_value(entity=entity,
                                    krpset_name="KRPset_KDS 14 20 54_4.3.5 (1)",
                                    en_name="Basic bond strength of a single bonded anchor in tension")
            fIpsecNa = fOpsecNa # KDS142054_040305_03 실행 결과
            fIn = self._get_p_value(entity=entity,
                                krpset_name="KRPset_KDS 14 20 54_4.3.5 (1)",
                                en_name="Number of Attachable Anchors")
            fIcNa = self._get_p_value(entity=entity,
                                    krpset_name="KRPset_KDS 14 20 54_4.3.5 (1)",
                                    en_name="The distance from the center of the anchor to the edge of the projected influence area required to develop the maximum bond strength")
            fIda = self._get_p_value(entity=entity,
                                    krpset_name="KRPset_KDS 14 20 54_4.3.5 (1)",
                                    en_name="The outer diameter of an anchor, or the shaft diameter of a head stud, head bolt, or claw bolt")
            fItauncr = self._get_p_value(entity=entity,
                                        krpset_name="KRPset_KDS 14 20 54_4.3.5 (1)",
                                        en_name="Characteristics Adhesion strength of adhesive anchors used in uncracked concrete")
            fIuserdefined = 1
            result = self.runit['01'].nominal_bond_strength_of_a_single_bonded_anchor(fINa,fINag,fIANa,fIANao,fIpsedNa,fIpscpNa,fINba,fIpsecNa,fIn,fIcNa,fIda,fItauncr,fIuserdefined)

            print(result)
            print("process end.")
        except:
            print("Fail")

In [9]:
reader = IFCReader(ifc_filepath="ED_extension_usingToolkit_v2.ifc")
rule_units = RuleUnitController('ruleunits')
ruleifc = RuleIFC_KDS142054_01(reader=reader, rule_units=rule_units)

In [10]:
entity = ruleifc.retrieve_entities("3s45s2y_vBWuTHL$ihTpyq")[0]

In [11]:
print(entity["KRPset_KDS 14 20 54_4.3.5 (1)"]["The distance from the center of the anchor to the edge of the projected influence area required to develop the maximum bond strength"])
print(ruleifc._get_p_value(entity, "KRPset_KDS 14 20 54_4.3.5 (1)", 'The distance from the center of the anchor to the edge of the projected influence area required to develop the maximum bond strength'))

-
-


In [12]:
entity = ruleifc.reader.get_products('IFCBearing')[109]
ruleifc.process(entity)

Pass
process end.


In [19]:
def convert_code_format(code):
        replacements = {
            "①": "_01",
            "②": "_02",
            "③": "_03",
            "④": "_04",
            "⑤": "_05",
            "⑥": "_06",
            "⑦": "_07",
            "⑧": "_08",
            "⑨": "_09",
        }
        parts = code.split("_")
        prefix = parts[0].replace(" ", "")
        parts[1] = parts[1].split(".")
        middle = ""
        for x in parts[1]:
            if len(x) > 1:
               middle += x
            else:
                middle += "0" + x 
        # parts[1] = parts[1].replace(".", "")
        # middle = "".join(["0" + char for char in parts[1]])
        suffix = "0" + parts[2][1]
        if any(key in code for key in replacements):
            suffix += replacements[code[-1]]

        return f"{prefix}_{middle}_{suffix}"

In [ ]:
convert_code_format("KDS 14 20 22_4.1.3_(2)")

'KDS142022_040103_02'